In [ ]:
import os, time, sys, shutil, zipfile, string, pickle
from distutils.dir_util import copy_tree
from copy import deepcopy
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk, gensim
from gensim.utils import simple_preprocess

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

!pip install visdom
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.13.2

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Args
dataset_name = "AmazonDigitalMusic"
dataset_path = '/content/drive/MyDrive/CAML/' + dataset_name + '/' + dataset_name + ".zip"
split_save_path = "/content/drive/MyDrive/CAML/" + dataset_name + "/"

!git clone https://github.com/deepeshhada/CAML-Explainable-Recs.git

In [ ]:
'''
    Copy preprocessed data
'''
%cd /content/CAML-Explainable-Recs/
copy_tree(split_save_path, './data/')
%cd /content/

In [ ]:
'''
    Train CAML
'''
%cd /content/CAML-Explainable-Recs/
!python train_CAML.py --opt Adam --emb_size 50 --rnn_size 50 --use_lower 1 --batch-size 128 --allow_growth 1 --dev_lr 0 --rnn_layers 1 --num_proj 1 --dropout 0.8 --epochs 10 --gpu 0 --hdim 50 --lr 0.001 --trainable 1 --eval 1 --dev 1 --num_heads 2 --hard 1 --early_stop 5 --translate_proj 1 --eval_train 0 --dmax 20 --smax 30 --init 0.01 --init_type xavier --att_reuse 0 --gmax 30 --rnn_dim 400 --gen_lambda 1.0 --masking 1 --implicit 1 --average_embed 1 --word_gumbel 1 --data_prepare 1 --word_pooling MEAN --att_type BILINEAR --key_word_lambda 0.25 --word_aggregate MAX --feed_rating 0 --data_link ./data --log_dir cv/test
%cd /content/

In [ ]:
'''
    Test and Generate
    Note: Assign best model's ckpt path and meta before inference
'''
model_meta_path = '/content/CAML-Explainable-Recs/cv/test/A2_Amazon_Instant_Video/RAW_MSE_CAML_FN_FM/31:01:22:23:39/model_best.ckpt.meta'
model_path = '/content/CAML-Explainable-Recs/cv/test/A2_Amazon_Instant_Video/RAW_MSE_CAML_FN_FM/31:01:22:23:39/model_best.ckpt'

%cd /content/CAML-Explainable-Recs/
!python test_CAML.py --opt Adam --emb_size 50 --rnn_size 50 --use_lower 1 --batch-size 100 --allow_growth 1 --dev_lr 0 --rnn_layers 1 --num_proj 1 --dropout 0.8 --epochs 50 --gpu 0 --hdim 50 --lr 0.001 --trainable 1 --eval 1 --dev 1 --num_heads 2 --hard 1 --early_stop 5 --translate_proj 1 --eval_train 0 --dmax 20 --smax 30 --init 0.01 --init_type xavier --att_reuse 0 --gmax 30 --rnn_dim 400 --gen_lambda 1.0 --masking 1 --implicit 1 --average_embed 1 --word_gumbel 1 --data_prepare 1 --word_pooling MEAN --att_type BILINEAR --key_word_lambda 0.25 --word_aggregate MAX --feed_rating 0 --data_link ./data --len_penalty 2 --beam_size 4 --gen_dir gen_dir/ --gen_true_dir true_dir/ --model $model_path --model_meta $model_meta_path
%cd /content/

In [ ]:
'''
    Save true and generated reviews for further analysis
'''
%cd /content/CAML-Explainable-Recs/
copy_tree('./gen_dir', split_save_path + 'gen_dir/')
copy_tree('./true_dir', split_save_path + 'true_dir/')
%cd /content/